In [1]:
#Installing libraries
!pip install transformers diffusers torch transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [2]:
#Importing libraries
import tweepy
import yaml
from diffusers import StableDiffusionPipeline
import torch
from transformers import pipeline, set_seed, BlipProcessor, BlipForConditionalGeneration
import re
from PIL import Image

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
# Set up the Twitter connection using OAuth 2.0
def twitConnection(creds):
    consumer_key = creds['twitter']['consumer_key']
    consumer_secret = creds['twitter']['consumer_secret']
    access_token = creds['twitter']['access_token']
    access_secret = creds['twitter']['access_secret']

    client = tweepy.Client(
        consumer_key=consumer_key, consumer_secret=consumer_secret,
        access_token=access_token, access_token_secret=access_secret
    )

    return client

In [4]:
# Set up the Twitter connection using OAuth 1.0
def twitConnection_v1(creds):
    consumer_key = creds['twitter']['consumer_key']
    consumer_secret = creds['twitter']['consumer_secret']
    access_token = creds['twitter']['access_token']
    access_secret = creds['twitter']['access_secret']

    auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    return tweepy.API(auth)

In [5]:
# To get user info
def get_user_info(api):
    try:
        user = api.verify_credentials()
        if user:
            print("User ID:", user.id)
            print("User Name:", user.name)
            print("User Screen Name:", user.screen_name)
            print("User Description:", user.description)
            print("User Location:", user.location)
            print("User Followers Count:", user.followers_count)
            print("User Following Count:", user.friends_count)
            print("User Tweets Count:", user.statuses_count)
        else:
            print("Failed to retrieve user information.")
    except tweepy.TweepyException as e:
        print("Error:", e)


In [6]:
# To delete a tweet
def delete_tweet(client, tweet_id):
    try:
        response = client.delete_tweet(tweet_id)
        if response:
            print(f"Tweet with ID {tweet_id} deleted successfully.")
        else:
            print(f"Failed to delete tweet with ID {tweet_id}.")
    except tweepy.TweepyException as e:
        print("Error:", e)

In [7]:
# Generate an image using Stable Diffusion
def generate_image(prompt):
    model_id = "dreamlike-art/dreamlike-diffusion-1.0"
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe = pipe.to("cuda")

    image = pipe(prompt).images[0]

    # Save the image
    image_path = "generated_image.png"
    image.save(image_path)

    return image_path

In [8]:
# Load the text generation pipeline with the standard GPT-2 model
torch.manual_seed(0)
generator = pipeline('text-generation', model='openai-community/gpt2')


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
# Generate text using GPT-2 model
def generate_text(prompt, max_length=60, temperature=0.8, top_k=40, top_p=0.9, repetition_penalty=1.2, num_return_sequences=5):
    generated_texts = generator(
        prompt,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences,
    )
    return [text['generated_text'].strip() for text in generated_texts]

In [10]:
# Post-process the text to trim to the last complete sentence
def trim_to_last_complete_sentence(text):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    if sentences:
        return ' '.join(sentences[:-1]) if len(sentences) > 1 else sentences[0]
    return text

In [11]:
# Generate caption for an image using BLIP model
def generate_caption(image_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(device)

    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption


In [12]:
# Load credentials
creds = yaml.load(open('/content/drive/MyDrive/Colab Notebooks/social_credentials.yml'), Loader=yaml.FullLoader)

# Set up Twitter clients
client = twitConnection(creds)
client_v1 = twitConnection_v1(creds)

In [13]:
# User choice for posting type
post_type = input("Enter post type (text/image/both): ").strip().lower()

Enter post type (text/image/both): image


In [14]:
# Variables to store content
msg = ""
media_id = None

In [15]:
if post_type == "text" or post_type == "both":
    # Generate text content
    text_prompt = "Bitcoin is a,"
    generated_texts = generate_text(text_prompt, max_length=60)
    print("Generated Texts:", generated_texts)  # Print generated texts
    trimmed_texts = [trim_to_last_complete_sentence(text) for text in generated_texts]
    print("Trimmed Texts:", trimmed_texts)  # Print trimmed texts
    msg = trimmed_texts[0] if trimmed_texts else "Generated text content."



In [20]:
if post_type == "image" or post_type == "both":
    # Generate image based on prompt
    image_prompt = "An image of a stack of colorful books with an e-reader resting on top"
    image_path = generate_image(image_prompt)

     # Generate caption for the generated image
    caption = generate_caption(image_path)
    print("Generated Caption:", caption)  # Print generated caption

     # Upload image to Twitter
    media = client_v1.media_upload(filename=image_path)
    media_id = media.media_id

    if post_type == "image":
        msg = f"{caption}"

if post_type == "both":
    msg += f"\n\n{caption}"



Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Generated Caption: a stack of books on a table


In [21]:
# Check if the generated text exceeds the Twitter character limit
if len(msg) > 280:
    print("Generated text exceeds Twitter's character limit. Please shorten the text or split it into multiple tweets.")
else:
    # Post tweet
    if media_id:
        response = client.create_tweet(text=msg, media_ids=[media_id])
    else:
        response = client.create_tweet(text=msg)

    tweet_id = response.data['id']  # Get the tweet ID from the response

    print(f"Tweet with ID {tweet_id} posted successfully.")

Tweet with ID 1797577249161355547 posted successfully.


In [22]:
# Get user info
get_user_info(client_v1)


User ID: 1788840492190380032
User Name: Developer
User Screen Name: anjanarani15049
User Description: Lifelong Learner
User Location: India
User Followers Count: 5
User Following Count: 8
User Tweets Count: 19


In [20]:
# Delete tweet if desired
delete_tweet_id = input("Enter the Tweet ID you want to delete: ")
delete_tweet(client, delete_tweet_id)

Enter the Tweet ID you want to delete: 1797502006002933932
Tweet with ID 1797502006002933932 deleted successfully.
